# dqRunning
views

In [15]:
!pip install dataquality_bnr

Looking in indexes: http://artifactory.produbanbr.corp/artifactory/api/pypi/pypi-all/simple
     |████████████████████████████████| 1.6 MB 39.1 MB/s eta 0:00:01
You should consider upgrading via the '/opt/miniconda/bin/python3 -m pip install --upgrade pip' command.


### Set up a PySpark session
A biblioteca foi construida para ser utilizada com __PySpark__ e possibilitar '*testes unitarios dos dados*', executando validacoes qualitativas em datasets de larga escala.</br>
A integracao da sessao spark e a biblioteca depende apenas de duas configuracoes adicionais:


In [4]:
from pyspark.sql import SparkSession, Row
from dataquality_bnr.dqSupport import main as dqSup

spark = SparkSession\
        .builder\
        .config("spark.jars", dqSup.getDeequJar_path())\
        .config("spark.jars.excludes", dqSup.getDeequJar_excludes())\
        .getOrCreate()

In [11]:
##Caso queira limpar o diretório antes de iniciar outra vez o tutorial

#!hdfs dfs -rm -r /user/x266727/dataquality_bnr-docs/histogram_anomalyDetec/DQ/

22/01/26 17:05:59 INFO fs.TrashPolicyDefault: Moved: 'hdfs://nameservice1/user/x266727/dataquality_bnr-docs/views/DQ' to trash at: hdfs://nameservice1/user/x266727/.Trash/Current/user/x266727/dataquality_bnr-docs/views/DQ


In [6]:
from dataquality_bnr.dqRunning import main as dqRun

In [12]:
dqView_critical = {"viewName" : "dqView_critical",
                   "inputData": "yamlFiles/views/critical_check/inputData.yaml",
                   "infraYaml": "yamlFiles/views/critical_check/infrastructure.yaml",
                   "vsYaml": "yamlFiles/views/critical_check/verificationSuite.yaml"}

dqView_moderate = {"viewName" : "dqView_check_moderate",
                   "inputData": "yamlFiles/views/moderate_check/inputData.yaml",
                   "infraYaml": "yamlFiles/views/moderate_check/infrastructure.yaml",
                   "vsYaml": "yamlFiles/views/moderate_check/verificationSuite.yaml"}

dqView_ad = {"viewName" : "dqView_ad",
             "inputData": "yamlFiles/views/basic_anomalyDetec/inputData.yaml",
             "infraYaml": "yamlFiles/views/basic_anomalyDetec/infrastructure.yaml",
             "vsYaml": "yamlFiles/views/basic_anomalyDetec/verificationSuite.yaml"}

In [13]:
myDq = dqRun.Dq(spark, "/user/x266727/dataquality_bnr-docs/views/DQ/")

myDq = (myDq
        .addView(dqView_critical, "Check")
        .addView(dqView_moderate, "Check")
        .addView(dqView_ad, "AnomalyDetection"))
        
dq_run_return = myDq.run()

__Evaluate DataQuality View and add to Main DataQuality Process__
dqView_critical pattern evaluated.
__Evaluate DataQuality View and add to Main DataQuality Process__
dqView_check_moderate pattern evaluated.
__Evaluate DataQuality View and add to Main DataQuality Process__
dqView_ad pattern evaluated.


__Running all DataQuality views__
run() dqView_critical:
getDataframe...
getCheck...
getOuputDFs...
writeTo_persistentRepository...
writeTo_currentResults...
DataQuality process finished.

run() dqView_check_moderate:
getDataframe...
getCheck...
getOuputDFs...
writeTo_persistentRepository...
writeTo_currentResults...
DataQuality process finished.

run() dqView_ad:
getPersistentRepository...
applyFiltering...
buildTemporaryRepository...
getDataframe...
getAnomalyDetection...
getOuputDFs...
As AnomalyDetection temporaryRepository is still empty. Forcing Success Outcome
writeTo_persistentRepository...
writeTo_currentResults...
DataQuality process finished.


__Get overall result__
writing 

In [16]:
dq_run_return.get_overall_booleanResult()

True

In [17]:
csvResult_df = dq_run_return.get_overall_csvResult_df(spark)
csvResult_df.toPandas()

,check,check_level,check_status,constraint,constraint_status,constraint_message,dataset_date,YY_MM_DD,viewName,viewPath
0,Cheking Metrics of Moderate severity,Warning,Warning,ComplianceConstraint(Compliance(i1c_cli_possui...,Failure,Value: 0.6206814369221901 does not meet the co...,1643227581826,2022-01-26,dqView_check_moderate,/user/x266727/dataquality_bnr-docs/views/DQ/dq...
1,Follow Table Size Changes,Warning,Success,AnomalyConstraint(Size(None)),Failure,Can't execute the assertion: requirement faile...,1643227592337,2022-01-26,dqView_ad,/user/x266727/dataquality_bnr-docs/views/DQ/dq...


#### pydeequ shutdown_callback_server()
#### spark.stop()
__Importante!__
Após a execucao dos jobs, garanta que a sessao __spark__ juntamente com o __callback_server__ sejam encerrados, evitando que qualquer processo "fantasma" fique pendurado.<br>
Leia mais sobre __Pydeequ__ e __callback_server__ em: https://github.com/awslabs/python-deequ

In [ ]:
spark.sparkContext._gateway.shutdown_callback_server()
spark.stop()